In [1]:
#%tensorflow_version 1.x

TensorFlow 1.x selected.


## Transfer Learning: Train some layers while freeze others

1. Import necessary libraries
2. Load the data
3. Pre-Process the data
4. Load weights of pretrained model and freeze few layers
5. Train the model
6. Get predictions

## 1. Import neccessary libraries

Here we will import the required libraries and functions.

In [8]:
# import necessary libraries for plotting images
import numpy as np
import matplotlib.pyplot as plt
# plot in jupyter notebook itself
%matplotlib inline
from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.models import Model
from keras.utils import to_categorical
#used to preprocess data according to VGG16
from keras.applications.vgg16 import preprocess_input
#for instantiating the model and loading the weights and biases
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, InputLayer

import tensorflow as tf

## 2. Load the data

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
# unzipping the Dataset.zip file
# !unzip /content/drive/My\ Drive/Dataset.zip

In [4]:
#reading the csv file containing data labels
data = pd.read_csv('../data/emergency_vehicles/emergency_classification.csv')

In [5]:
# create an empty python list
X = []

# go through all the image locations one by one
for img_name in data.image_names:
    # read the image from location
    img = plt.imread('../data/emergency_vehicles/images_vehicles/' + img_name)
    # pile it one over the other
    X.append(img)
    
# convert this python list to a single numpy array
X = np.array(X)

In [6]:
#getting the labels for images
y = data.emergency_or_not.values

## 3. Pre-Process the data

Steps : 
1. Pre-process the data as per model's requirement
2. Prepare training and validation set

In [9]:
#preprocess input images accordiing to requirements of VGG16 model
# X = preprocess_input(X, mode='tf')
X = tf.keras.applications.imagenet_utils.preprocess_input(X, mode='tf') #tf scales data -1 to 1

In [10]:
#converting label to categorical i.e instead of 0/1 labels we have 2 columns emergency and non-emergency
# with only one of them is true for every image
y = to_categorical(y)

In [11]:
# splitting the dataset into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

## 4. Load weights of pretrained model and freeze few layers

In [12]:
base_model = VGG16(weights='imagenet')

In [12]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [14]:
# creating our own model 
x = Dense(100, activation='relu', name='fc1')(base_model.layers[-4].output)
y = Dense(2, activation='softmax', name='prediction')(x)
my_model = Model(base_model.input, y)
my_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [15]:
my_model.layers

In [16]:
len(my_model.layers)

22

In [17]:
#To set the first 15 layers to non-trainable (weights will not be updated)
for layer in my_model.layers[:15]:
    layer.trainable = False 

In [18]:
# compile the model
my_model.compile(optimizer='sgd', metrics=['accuracy'], loss='categorical_crossentropy')

## 5. Training the model

Freezing the layers can save substabtial processing time.

In [19]:
# training the model
my_model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
52/52 [==============================] - 360s 7s/step - loss: 0.5668 - accuracy: 0.7491 - val_loss: 0.2644 - val_accuracy: 0.8810
Epoch 2/10
52/52 [==============================] - 494s 9s/step - loss: 0.2978 - accuracy: 0.8846 - val_loss: 0.2233 - val_accuracy: 0.9079
Epoch 3/10
52/52 [==============================] - 486s 9s/step - loss: 0.1520 - accuracy: 0.9459 - val_loss: 0.2151 - val_accuracy: 0.9122
Epoch 4/10
52/52 [==============================] - 487s 9s/step - loss: 0.1363 - accuracy: 0.9465 - val_loss: 0.1484 - val_accuracy: 0.9448
Epoch 5/10
52/52 [==============================] - 486s 9s/step - loss: 0.0655 - accuracy: 0.9806 - val_loss: 0.1643 - val_accuracy: 0.9249
Epoch 6/10
52/52 [==============================] - 474s 9s/step - loss: 0.0373 - accuracy: 0.9927 - val_loss: 0.1311 - val_accuracy: 0.9490
Epoch 7/10
52/52 [==============================] - 498s 10s/step - loss: 0.0276 - accuracy: 0.9939 - val_loss: 0.1611 - val_accuracy: 0.9320
Epoch 8/10
5

## 6. Get Predictions

In [ ]:
# get predictions
predictions = my_model.predict(X_valid)
predicted_class = np.argmax(predictions,axis=1)

In [20]:
# checking the performance of model on validation set
accuracy_score(y_valid[:,1], predicted_class)

0.9546742209631728

## Steps to use transfer learning where we train some layers while freeze others

1. Import necessary libraries
2. Load the data
3. Pre-Process the data
4. Load weights of pretrained model and freeze few layers
5. Train the model
6. Get predictions